****SESIÓN 7 RECOMENDADORES DE MÚSICA****

**ALUMNOS:** Ignacio García Fernández, Jorge Torres Ruiz
Grupo 11

Vamos a construir sistemas recomendadores de música de varios tipos(popularidad, géneros, filtrado colaborativo..)
Posteriormente analizaremos cada uno de ellos en detalle

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Cargamos los datos y realizamos un análisis exploratorio de los mismos

In [2]:
ratings = pd.read_csv('ratings_songs.csv')
songs = pd.read_csv('songs.csv')

Breve visión del contenido de los archivos y de información de valor:

In [3]:
songs.head()

,id,title,artist,genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,1,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83
1,2,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82
2,3,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80
3,4,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79
4,5,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78


In [4]:
ratings.head()

,userId,id,rating,timestamp
0,1,28443,5,1701024397
1,1,13087,1,1701039432
2,1,25095,0,1701033109
3,1,28737,4,1701046306
4,1,27284,0,1701041798


In [5]:
ratings = ratings.loc[ratings['id'] <= 603]

In [6]:
n_ratings = len(ratings)
n_songs = ratings['id'].nunique()
n_users = ratings['userId'].nunique()
print(f"Numero de ratings: {n_ratings}")
print(f"Numero de songId's: {n_songs}")
print(f"Numero de users: {n_users}")
print(f"Número medio de ratings por usuario: {round(n_ratings/n_users, 2)}")
print(f"Número medio de ratings por canción: {round(n_ratings/n_songs, 2)}")
print(f"Rating global medio: {round(ratings['rating'].mean(),2)}.")
mean_ratings = ratings.groupby('userId')['rating'].mean()
print(f"Ratio medio por usuario: {round(mean_ratings.mean(),2)}.")

Numero de ratings: 53589
Numero de songId's: 604
Numero de users: 606
Número medio de ratings por usuario: 88.43
Número medio de ratings por canción: 88.72
Rating global medio: 2.49.
Ratio medio por usuario: 2.47.


### 1. Recomendador basado en popularidad
Vamos a hacer un primer recomendador básico basado en popularidad. 
Medimos la popularidad de las películas para saber cuál es la mejor y peor película, es decir, qué película tiene la media de ratings más alta y más baja. 

In [7]:
#obtenemos la media de rating agrupada pos película
media_ratings = ratings.groupby('id')[['rating']].mean()
media_ratings

,rating
id,
0,2.069444
1,2.641304
2,2.333333
3,2.523810
4,2.282051
...,...
599,2.538462
600,2.255814
601,2.428571


Como ejemplo mostramos la canción con mejor nota y la de peor nota

In [8]:
lowest_rated = media_ratings['rating'].idxmin()
songs[songs['id'] == lowest_rated]

,id,title,artist,genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
119,120,Glad You Came,The Wanted,boy band,2012,127,85,72,-4,11,45,198,3,6,72


In [9]:
highest_rated = media_ratings['rating'].idxmax()
songs[songs['id'] == highest_rated]
#ratings[ratings['songId'] == highest_rated]

,id,title,artist,genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
144,145,Roar,Katy Perry,dance pop,2013,180,77,55,-5,35,46,224,0,4,78


Nuestro recomendador (en este momento) solo tiene en cuenta la nota en sí de las canciones, es decir, tiene el mismo impacto una canción con una valoración de 4 que otra con nota de 4 tras 10.000 valoraciones lo cual no hace justicia al número de veces que se ha valorado una canción. Para ser más justos emplearenos la media bayesiana. 


$r_{i} = \frac{C \times m + \Sigma{\text{reviews}}}{C+N}$

Donde:
  - $C$ representa la confianza
  - $m$ representa el conocimiento previo (prior) que en este ejemplo será el rating medio (calculado con todas las películas). 
  - $N$ es el numero total de ratings para la película $i$ 
  - $C$ representa el tamaño típico de un data set. En este caso $C$ será el número medio de ratings de las películas. 

In [10]:
songs_stats = ratings.groupby('id')[['rating']].agg(['count', 'mean'])
songs_stats.columns = songs_stats.columns.droplevel()   #quitar titulo rating arriba
songs_stats
#AQUÍ MOSTRAMOS LA MEDIA DE NOTA JUNTO CON LAS VECES QUE SE VA VALORADO

,count,mean
id,,
0,72,2.069444
1,92,2.641304
2,93,2.333333
3,84,2.523810
4,78,2.282051
...,...,...
599,91,2.538462
600,86,2.255814
601,105,2.428571


In [11]:
#APLICAMOS LA FÓRMULA DE LA MEDIA BAYESIANA

C = songs_stats['count'].mean()
m = songs_stats['mean'].mean()

def bayesian_avg(ratings):
    bayesian_avg = (C*m+ratings.sum())/(C+ratings.count())
    return bayesian_avg

bayesian_avg_ratings = ratings.groupby('id')['rating'].agg(bayesian_avg).reset_index()
bayesian_avg_ratings.columns = ['id', 'bayesian_avg']
songs_stats = songs_stats.merge(bayesian_avg_ratings, on='id')

*Mostramos las mejores películas*

In [12]:
songs_stats = songs_stats.merge(songs[['id', 'title']])
songs_stats.sort_values('bayesian_avg', ascending=False).head()


,id,count,mean,bayesian_avg,title
61,62,98,3.030612,2.774259,What the Hell
144,145,81,3.061728,2.763432,Roar
581,582,90,2.988889,2.741773,Good as Hell (feat. Ariana Grande) - Remix
60,61,87,2.977011,2.731674,Tonight Tonight
102,103,79,2.936709,2.700989,Firework


*Mostramos las peores canciones*

In [13]:
songs_stats = songs_stats.merge(songs[['id', 'title']])
songs_stats.sort_values('bayesian_avg', ascending=True).head()

,id,count,mean,bayesian_avg,title
582,583,97,2.000000,2.234608,Higher Love
52,53,84,1.964286,2.234897,Someone Like You
219,220,78,1.948718,2.237353,Treasure
119,120,76,1.947368,2.240235,Glad You Came
338,339,75,1.986667,2.260026,Prayer in C - Robin Schulz Radio Edit


*Conclusiones:* Tras aplicar la media Bayesiana nuestro recomendador es mucho más crítico y tiene en cuenta el número de valoraciones de las canciones. En este tipo de recomendadores el peso reside en las vaoraciones unicamente, no se tienen en cuenta los gustos del usuario por ejemplo, lo cual veremos como solucionar en recomendadores posteriores.

### 2. Recomendador basado en contenidos usando similitud del coseno

Vamos a crear un recomendador que utilice la similitud del coseno para recomendar canciones en función de su género, la idea consiste en que si nos gusta una canción es porbable que las canciones pertenecientes al mismo o mismos géneros nos gusten también

In [14]:
numSongs = songs['id'].nunique()
print(f"Hay {numSongs} películas en el dataset.")

Hay 603 películas en el dataset.


Tenemos que crear una matriz binaria que representa si una película (fila) se encuentra dentro de un género (columna)

In [15]:
genres = set(g for G in songs['artist'] for g in G)

for g in genres:
    songs[g] = songs.artist.transform(lambda x: int(g in x))
    
song_genres = songs.drop(columns=['id','title','artist', 'genre', 'year', 'bpm', 'nrgy', 'dnce', 'dB', 'live', 'val', 'dur', 'spch', 'pop', 'acous', 'artist'])
song_genres

,E,n,M,p,S,d,U,é,w,H,...,D,f,T,o,j,J,C,P,s,t
0,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,0,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
599,1,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
600,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
601,0,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


Ahora vamos a crear una matriz de similitud entre los géneros con la ayuda de la similitud del coseno

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(song_genres, song_genres)
print(f"Dimensiones de la matriz de similitud del coseno entre los géneros: {cosine_sim.shape}")

Dimensiones de la matriz de similitud del coseno entre los géneros: (603, 603)


In [17]:
cosine_sim


array([[1.        , 0.4       , 0.2       , ..., 0.1490712 , 0.4472136 ,
        0.62017367],
       [0.4       , 1.        , 0.2       , ..., 0.1490712 , 0.1490712 ,
        0.49613894],
       [0.2       , 0.2       , 1.        , ..., 0.59628479, 0.2981424 ,
        0.49613894],
       ...,
       [0.1490712 , 0.1490712 , 0.59628479, ..., 1.        , 0.22222222,
        0.36980013],
       [0.4472136 , 0.1490712 , 0.2981424 , ..., 0.22222222, 1.        ,
        0.64715023],
       [0.62017367, 0.49613894, 0.49613894, ..., 0.36980013, 0.64715023,
        1.        ]])

La diagonal siempre está formada por 1s, un género siempre es 100% similar consigo mismo

A continuación creamos una función que nos permite encontrar títulos que contengan una cadena de caractereres pasada como argumento

In [18]:
def song_finder(title):
     return songs[songs['title'].str.contains(title)]['title'].tolist()

song_finder('Baby')

['Baby', "Baby Don't Lie"]

Ahora se calculan dada una canción cuales son las canciones que más similitud presentan con respecto a esta y por tanto las que se recomiendan.

In [19]:

def mostSimilarSongs(song):
    movie_idx = dict(zip(songs['title'], list(songs.index)))
    title = song_finder(song)[0]
    n_recommendations = 5

    idx = movie_idx[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:(n_recommendations+1)]
    similar_songs = [i[0] for i in sim_scores]
    score_similarSongs = [i[1] for i in sim_scores]

    print(f"Recomendaciones similares a {title} según la similitud entre sus géneros:")
    recommendations_df = songs.iloc[similar_songs][['title', 'genre']]  # Seleccionar las columnas 'title' y 'genre'
    recommendations_df['similarity_score'] = score_similarSongs  # Agregar los scores de similitud

    print(recommendations_df)


La siguiente función nos permite calcular la media de similitud entre las x que se seleccionan como más similares con respecto a una dada

In [23]:
def mostSimilarSongsMean(song):
    movie_idx = dict(zip(songs['title'], list(songs.index)))
    title = song_finder(song)[0]
    n_recommendations = 10

    idx = movie_idx[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:(n_recommendations+1)]
    similar_songs = [i[0] for i in sim_scores]
    score_similarSongs = [i[1] for i in sim_scores]
    
    mean = 0
    for score in score_similarSongs:
        mean+= score
    mean/= n_recommendations
    return mean




In [24]:
mostSimilarSongs('Cooler Than Me - Single Mix')

Recomendaciones similares a Cooler Than Me - Single Mix según la similitud entre sus géneros:
                                               title      genre  \
84                                   Please Don't Go  dance pop   
380              I Took A Pill In Ibiza - Seeb Remix  dance pop   
272                                      Uptown Funk  dance pop   
598              Find U Again (feat. Camila Cabello)  dance pop   
601  Nothing Breaks Like a Heart (feat. Miley Cyrus)  dance pop   

     similarity_score  
84           1.000000  
380          1.000000  
272          0.737865  
598          0.737865  
601          0.737865  


Para la canción Cooler than me las canciones que se recomiendan debido a su similitud en cuanto al género son las que se muestran encima. 
No obstante puedo asegurar que es es bastande preciso ya que he lanzado el algoritmo con una canción que me gusta y los resultados son los esperados.
Además se puede comprobar que la tabla selecciona canciones en orden descendiente en cuanto a similitud obteniendo primero las más similares

In [22]:
allSongsMean = 0;
l = songs['title'][:8]
for z in l:
    allSongsMean+= mostSimilarSongsMean(z)
print('De una muestra de 8 canciones se calculan sus 5 canciones más similares')
print('Haciendo la media de los scores de similitud para cada canción con respecto a las 8 seleccionadas hemos obtenido que:')
print('La media de similitud es de:')
print(allSongsMean/8)
    

De una muestra de 8 canciones se calculan sus 5 canciones más similares
Haciendo la media de los scores de similitud para cada canción con respecto a las 8 seleccionadas hemos obtenido que:
La media de similitud es de:
0.8566805677094819


 ### 3. Recomendador colaborativo user-user usando similitud de Pearson

En esta versión vamos a implementar la medida de similitud de Pearson para obtener los usuarios más similares. 
En esta versión la recomendación puede usar las preferencias de usuarios parecidos calculando para ello la similitud entre sus vectores de valoraciones. 

In [25]:
def getUserVotes(user):
    return ratings.loc[ratings['userId'] == user, 'rating'].values

def getVotedSongs(user):
    return ratings.loc[ratings['userId'] == user, 'id'].values

def getRating(userId, id):
    data  = ratings.loc[(ratings['userId'] == userId) & (ratings['id'] == id), 'rating']
    if len(data)==0:
        return 0
    else:
        return data.iloc[0]
    
def getNormalizedRating(userId, id):
    rating = getRating(userId, id)
    if rating == 0:
        return rating
    else:
        return rating-getUserVotes(userId).mean()

In [26]:
## Calcula la similitud de dos usuarios dados.
def sim(a,u):
    a_votedSongs = getVotedSongs(a)
    u_votedSongs = getVotedSongs(u)

    a_votes  = getUserVotes(a)
    u_votes  = getUserVotes(u)

    a_mean = a_votes.mean()
    u_mean = u_votes.mean()

    commonVotedSongs = [value for value in a_votedSongs if value in u_votedSongs]
    s = len(commonVotedSongs)
    if s==0:
        return 0
    
    a_votes = [getRating(a, id) for id in commonVotedSongs]
    u_votes = [getRating(u, id) for id in commonVotedSongs]
    a_votes_norm = [v - a_mean for v in a_votes]
    u_votes_norm = [v - u_mean for v in u_votes]
    
    dot = np.multiply(a_votes_norm, u_votes_norm)
    sum_dot = np.sum(dot)
    std  = (np.std(a_votes) * np.std(u_votes))
    if std==0:
        std = 1
    
    f  = 50 #Debería ser la media de peliculas en común de todo el dataset
    sim = sum_dot * s / std / f
    return sim

In [27]:
# Recupera k usuarios más similares al usuario con idUSer=a
def getKMostSimilarUsers(a,k):
    users = ratings.userId.unique()
    similarity = [sim(a,u) for u in users]
    data = {'User': users,
        'Similarity': similarity}
    simTable = pd.DataFrame(data)
    simTable = simTable[simTable.User !=a]
    simTable = simTable.sort_values('Similarity',ascending=False)
    return simTable.head(k)    

In [36]:
getKMostSimilarUsers(1,5)

,User,Similarity
599,604,1.683197
39,41,1.626027
515,520,1.617560
500,505,1.575433
295,300,1.256398
